## 1-minute introduction to Jupyter ##

A Jupyter notebook consists of cells. Each cell contains either text or code.

A text cell will not have any text to the left of the cell. A code cell has `In [ ]:` to the left of the cell.

If the cell contains code, you can edit it. Press <kbd>Enter</kbd> to edit the selected cell. While editing the code, press <kbd>Enter</kbd> to create a new line, or <kbd>Shift</kbd>+<kbd>Enter</kbd> to run the code. If you are not editing the code, select a cell and press <kbd>Ctrl</kbd>+<kbd>Enter</kbd> to run the code.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Lesson 12a: Class inheritance, local imports

In [ ]:
# This code cell loads a PEP8 linter
# Linting is the process of flagging programming errors,
# bugs, stylistic errors, and other code problems
# pycodestyle is a linter that highlights any syntax that
# is not PEP8-compliant.

%load_ext pycodestyle_magic
%pycodestyle_on

## Class Inheritance

While `class`es are really useful, we will find ourselves repeating a lot of code if we don’t think through and plan the code in advance. Let’s try writing a simple chess game. Before we start writing any code, it helps immensely to think about what we will need first.

At the least, we will need:

- a chess board
- chess pieces
  - king
  - queen
  - rook
  - bishop
  - knight
  - pawn
  
The pieces have one common characteristics:

- a colour (white or black)

6 different types of pieces. That means repeating our `__init__()`, `__repr__()`, and other methods _6 times_! And if we change our minds and decide to change one of those features, we will have to change it in 6 places … Is there any way we can reduce that repetition?

Of course. Python lets us define classes _based on other classes_.

Since all chess pieces have some common characteristics, let’s define a `BasePiece` class that represents a _generic_ chess piece. It will implement code that is common to all chess pieces:

In [ ]:
class BasePiece:
    def __init__(self, colour):
        if type(colour) != str:
            raise TypeError('colour argument must be str')
        elif colour.lower() not in {'white', 'black'}:
            raise ValueError('colour must be {white, black}')
        else:
            self.colour = colour

    def __repr__(self):
        return f'BasePiece({repr(self.colour)})'

## Child classes

Our child classes should have the attributes of the `BasePiece`, which is the parent class. We call them child classes because they are derived from the parent class. We say that the child class **inherits** the attributes and methods of the parent class.

Let’s start with our first child class, the `King`:

In [ ]:
class King(BasePiece):
    pass

This child class inherits the `colour` attribute, as well as the `__init__()` and `__repr__()` methods from `BasePiece`.

In [ ]:
k = King('white')
k

## Class attributes

Hmm, that’s not so helpful now. Our `BasePiece` only needed a `colour` since it was a generic piece, but now that we are creating pieces of a specific type, `__repr__()` should also return the piece name.

We will need to **override** the `__repr__()` method of `BasePiece` by defining a new one for `King`. But how do we create a `name` attribute for `King` without going through the `__init__()` method?

We can set it as a **class attribute** instead.

Notice that the `colour` attribute of `BasePiece` was set only in `__init__()`. The `BasePiece` class **does not** actually have this attribute, only its instances have it:

In [ ]:
b = BasePiece('white')
print(f'b.colour: {b.colour}')
print(f'BasePiece.colour: {BasePiece.colour}')

The `King` class, on the other hand, represents a `king` piece, whether it has been instantiated or not. I should be able to do this:

    >>> King.name
    'king'
    
So let’s go ahead and give the `King` class a `name` class attribute, and a new `__repr__()` method.

In [ ]:
class King(BasePiece):
    # define a `name` class attribute for the King class
    name = 'king'

    def __repr__(self):
        # define the __repr__() method for the King class
        


k = King('white')
print(k)

In [ ]:
# AUTOGRADING: test class attribute and __repr__()
assert King.name == 'king', \
    '`name` attribute wrongly defined for King class'
assert repr(King('white')) == "King('white')", \
    "__repr__() method wrongly defined for King class"

### Task 1

Later, we are going to need a `__str__()` method to produce a simple description (e.g. `'white king'`) too. That’s a simple combination of the `colour` and `name` attributes, and it would be tedious to repeat the `__str__()` definition for all the piece classes.

So let’s define it in `BasePiece` instead, using `try-except` to catch the `NameError` if the `name` attribute is not found:

In [ ]:
class BasePiece:
    def __init__(self, colour):
        if type(colour) != str:
            raise TypeError('colour argument must be str')
        elif colour.lower() not in {'white', 'black'}:
            raise ValueError('colour must be {white, black}')
        else:
            self.colour = colour

    def __repr__(self):
        return f'BasePiece({repr(self.colour)})'

    def __str__(self):
        try:
            # define __str__() to return a simple description
            # e.g. 'white king', 'black queen'
            
        except NameError:
            return f'{self.colour} piece'


class King(BasePiece):
    name = 'king'

    def __repr__(self):
        return f'King({repr(self.colour)})'


k = King('white')
print(k)

In [ ]:
# Test cell to check your code
assert str(King('white')).strip() == 'white king', \
    "__str__() method wrongly defined for King class"

There, `King` is working much better now.

## Instance attributes and attribute overriding

Wait, why is `self.name` able to be used when we didn’t set it in `__init__()`? That’s because class attributes are available to all its instances. If you set the `name` attribute of an instance, it will override its class attribute (note that the class attribute is not deleted). And if you delete the instance attribute, the class attribute will be used again:

In [ ]:
k.name = 'da king'
# Uses instance attribute
print(f'After overriding class attribute: {k.name}')
del k.name
# Back to class attribute
print(f'After removing instance attribute: {k.name}')

Lets go ahead and make `Board` first, before we come back to look at the other pieces.

## Making `Board`

Our chess board needs to have an 8×8 grid. It also needs to have a way to keep track of which piece is on which square of the grid. How should we go about doing this?

A newcomer might think of creating an 8-by-8 nested list, like this:

In [ ]:
board = []
for x in range(8):
    none_row = [None]*8
    board.append(none_row)
board[0][4] = King('black')
board[7][4] = King('white')
board

But then you are going to have a hard time tracking their positions; how are you going to find `King('black')` after it has moved?

    for x in range(8):
        for y in range(8):
            piece =  board[x][y]
            if piece.name == 'king' and piece.colour = 'black':
                <your code goes here>

That could work, but it’s so inefficient. You have 64 board positions, and 32 board pieces. How complex is your code going to have to be?

It would be easier instead to just store the positions of the pieces. Lets think about our ideal code. We would like to be able to get the positions of each chess piece this way:

    >>> b = Board() # instantiates a game board
    <-- Suppose our chess pieces have been placed on the board at this point -->
    >>> b.position_of_piece('white king') # indexes start from 0
    (7, 4)
    
It would also be helpful if we could examine which piece was at a particular position:

    >>> b.piece_at_position(7, 4)
    'white king'
    
Then it looks like each piece is **mapped** to a position. Sounds familiar? Perhaps we could use a `dict` to map each chess piece to a position!

Oh man, there’s no easy way around this. We are going to have to write code to set the initial position of each piece.

### Task 2

Before you run the cell below, time to define the other classes first so this will work:

In [ ]:
# Define the other chess piece classes and their reprs here:




Now that we have our pieces, we can start to set them up on the playing board. We will map each piece to a specific position on the board. `dict`s accept any immutable object as a key, so we will use tuples as the key for each piece; the piece object itself is the value.

We also need a method, `display()`, to print the entire board, so that we can see what is going on.

In [ ]:
# Run this cell to update `Board` and generate
# a playing position with starting positions


class Board:
    def __init__(self):
        pass
    
    def start(self):
        self.position = []

        colour = 'black'
        self.position[(0, 7)] = Rook(colour)
        self.position[(1, 7)] = Knight(colour)
        self.position[(2, 7)] = Bishop(colour)
        self.position[(3, 7)] = Queen(colour)
        self.position[(4, 7)] = King(colour)
        self.position[(5, 7)] = Bishop(colour)
        self.position[(6, 7)] = Knight(colour)
        self.position[(7, 7)] = Rook(colour)
        for x in range(0, 8):
            self.position[(x, 6)] = Pawn(colour)

        colour = 'white'
        self.position[(0, 0)] = Rook(colour)
        self.position[(1, 0)] = Knight(colour)
        self.position[(2, 0)] = Bishop(colour)
        self.position[(3, 0)] = Queen(colour)
        self.position[(4, 0)] = King(colour)
        self.position[(5, 0)] = Bishop(colour)
        self.position[(6, 0)] = Knight(colour)
        self.position[(7, 0)] = Rook(colour)
        for x in range(0, 8):
            self.position[(x, 1)] = Pawn(colour)

    def display(self):
        '''
        Displays the contents of the board.
        Each piece is represented by two letters.
        First letter is the colour (W for white, B for black).
        Second letter is the name (Starting letter for each piece).
        '''
        # Write your code here
        

b = Board()
b.position

That’s all the _information_ we need about the pieces. But we will need more _methods_ in the process of programming the chess game. We will continue that in **Lesson 12b**.

For now, we have quite a lot of code scattered across many cells. Let’s put them all into a single file so that it is easier to manage. In Python, this is known as making a **module**.

## Task 1: Making a `chess` (single-file) module

Copy the latest code for `Board`, `BasePiece`, and each chess piece class (`King`,`Queen`,`Bishop`,`Knight`,`Rook`,`Pawn`) into `chess.py`, overriding the old definitions.

## Local imports

Besides the standard Python libraries, you can also install other libraries. The most common way of doing this is through the <b>P</b>ackage <b>I</b>nstaller for <b>P</b>ython, also known as `pip`. It acts like an “app store” for Python, except it has python libraries instead of apps.

You can’t run `pip` on the school laptops as you don’t have administrator permissions, but on your own laptop you can do so. We will look at `pip` use once we begin on group-based projects. For now, let’s look at a related concern: how do you import a library you wrote yourself, but which is not available on `pip`?

## Importing from another Python file (`.py`) in the same directory

A library can be very simple; nothing more than another `.py` file containing functions and classes. It can also be very complex, consisting of multiple layers of files and directories, possibly even requiring installation.

We have just created a`chess` module, inside `chess.py`, in the same directory as this Jupyter Notebook.
Let’s import those objects into this notebook.

In [ ]:
from chess import Board, BasePiece, King, Queen, Bishop, Knight, Rook, Pawn

b = Board()
b.field

For modules with many more objects, it can get tedious to list every single class and function. In those cases, to keep the names clear (remember how hard it is to name things?), we simply import the module. The classes (and any functions) are available with the `module.class` (or `module.function`) syntax:

In [ ]:
import chess

board = chess.Board()
board.field

Very similar to your normal `import`s, right?

Python will first search in the directory to see if there is a file or library named `chess`. If it doesn’t exist, then Python will search in a list of directories to see if there are any libraries or modules named `chess`. This list of directories is known as the **system path**.

In [ ]:
import sys

print('Directories in system path:')
for path in sys.path:
    print(path)

If nothing named `chess` is found in any of these places, Python raises a `ModuleNotFoundError`. For example:

In [ ]:
import chess2

# Feedback and suggestions

Any feedback or suggestions for this assignment?

YOUR ANSWER HERE